In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml calibration_config

experiment:
  seed: 42
    
data:
  preload: False 
  iters_per_epoch: 500

dataloader:
  batch_size: 1 
  num_workers: 1
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 

train:
  epochs: 300 
  eval_freq: 10 
  pretrained_dir: /storage/vbutoi/scratch/ESE/training/10_23_23_Dense_WMH 

log:
  checkpoint_freq: 50 
  root: '?'
  metrics:
    ece_loss:
      _fn: ese.experiment.metrics.ece_loss
      from_logits: True
    edge_ece_loss:
      _fn: ese.experiment.metrics.edge_ece_loss
      from_logits: True
    elm_loss:
      _fn: ese.experiment.metrics.elm_loss
      from_logits: True

loss_func: 
  _class: ionpy.loss.PixelCELoss 
  from_logits: True 
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibrator_model_cfg  

model:
  _class: '?'
  num_classes: 2
  image_channels: 1

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

<IPython.core.display.Javascript object>

In [5]:
# Setup direcrtories
calibration_root = '/storage/vbutoi/scratch/ESE/calibration'

# WMH CONFIG
exp_name = '01_03_24_WMH_LongerCalibrationSuite'

# Setup the root.
exp_root = f'{calibration_root}/{exp_name}'

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'optim.lr': [1e-4],
        'model._class': ['ese.experiment.models.calibrators.Temperature_Scaling',
                         'ese.experiment.models.calibrators.Vector_Scaling',
                         'ese.experiment.models.calibrators.Dirichlet_Scaling',
                         'ese.experiment.models.calibrators.LTS',
                         'ese.experiment.models.calibrators.Selective_Scaling'],
    },
]

In [6]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config

# Assemble base config
base_cfg = Config(calibration_config).update([calibrator_model_cfg, callbacks_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [7]:
len(cfgs)

5

## Running Jobs

In [8]:
from ese.experiment.experiment import run_ese_exp, submit_ese_exps, PostHocExperiment

In [9]:
# Run individual jobs
run_ese_exp(
    config=cfgs[1], 
    experiment_class=PostHocExperiment,
    gpu='0',
    run_name='debug',
    show_examples=False,
    track_wandb=True
)

Random seed set as: 42


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Random seed set as: 42
Running PostHocExperiment("/storage/vbutoi/scratch/ESE/calibration/debug/20240104_155906-CP5Q-377dc7e91d9784eef03ebbbaf8f64c7d")
---
callbacks:
  epoch:
  - ese.experiment.callbacks.WandbLogger
  - ionpy.callbacks.ETA
  - ionpy.callbacks.JobProgress
  - ionpy.callbacks.TerminateOnNaN
  - ionpy.callbacks.PrintLogged
  - ionpy.callbacks.ModelCheckpoint:
      monitor: ece_loss
      phase: val
data:
  iters_per_epoch: 500
  preload: false
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  seed: 42
log:
  checkpoint_freq: 50
  metrics:
    ece_loss:
      _fn: ese.experiment.metrics.ece_loss
      from_logits: true
    edge_ece_loss:
      _fn: ese.experiment.metrics.edge_ece_loss
      from_logits: true
    elm_loss:
      _fn: ese.experiment.metrics.elm_loss
      from_logits: true
  root: /storage/vbutoi/scratch/ESE/calibration/debug
  wandb_string: exp_name:01_03_24_WMH_LongerCalibrationSuite-lr:0.0001-_class:ese.experiment.models.cali

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [ ]:
# # Run Batch Jobs
# submit_ese_exps(
#     exp_root=exp_root,
#     experiment_class=PostHocExperiment,
#     config_list=cfgs,
#     available_gpus=['0', '1', '2', '3'],
#     track_wandb=True
# )